## Libraries

In [ ]:
!pip install sqlalchemy

In [ ]:
!pip install folium

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install geopy

In [ ]:
!pip install psycopg2

In [7]:
import sqlalchemy as db
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import folium
import time
from IPython.display import HTML
import datetime
from geopy.distance import geodesic
from dotenv import load_dotenv
import os

In [8]:
load_dotenv()
password = os.getenv('PASSWORD')

## Connection to database 

> It creates a connection with the database on Timescale


In [13]:
engine = db.create_engine('postgresql://tsdbadmin'+password+'@jguo00m642.xacmv54b1t.tsdb.cloud.timescale.com:34972/tsdb')

In [ ]:
connection = engine.connect()
metadata = db.MetaData()
data = db.Table('dados_torque', metadata, autoload=True, autoload_with=engine)

## Results from database

### Columns

In [ ]:
print(data.columns.keys())

### Data

In [ ]:
query = db.select([data])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet[-3:-1]

## Organizing data

In [ ]:
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
print(df.shape)
df.head()

## Information in graphs

In [ ]:
user_email = 'youremail@email.com' #Put your e-mail here
day = '2023-04-07' #Put the day of the trip here (format yyyy-mm-dd)
day_to_filter = pd.Timestamp(day).date()

df_filter = df[df['appid_userentered'] == user_email] 
df_filter = df_filter[df_filter['time_sec'].dt.date == day_to_filter] 

print(df_filter.shape)
df_filter.head()

In [ ]:
df_filter.dropna(axis=1, inplace=True)
df_filter.head()

### Part 1) Speed values

#### All speed sensor records

In [ ]:
time = df_filter['time_sec']
speed = df_filter['speedobd']

fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=speed, mode='lines', name='Speed'))

fig.update_layout(
    title={
        'text': 'Speed values for a trip',
        'font': {'size': 24}
    },
    xaxis_title='Time',
    yaxis_title='Speed (km/h)',
    font=dict(family='Arial', size=18, color='black'),
    plot_bgcolor='rgb(250, 250, 250)',
    paper_bgcolor='rgb(250, 250, 250)'
)

fig.update_traces(line=dict(width=2, color='RoyalBlue'))

fig.update_xaxes(
    showgrid=True,
    gridcolor='lightgrey',
    gridwidth=0.5,
    tickfont=dict(family='Arial', size=14, color='black')
)

fig.update_yaxes(
    showgrid=True,
    gridcolor='lightgrey',
    gridwidth=0.5,
    tickfont=dict(family='Arial', size=14, color='black')
)

fig.show()

#### Average trip speed

In [ ]:
avg_speed = round(speed.mean(), 2)
min_speed = min(speed)
max_speed = max(speed)

fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=avg_speed,
    title={'text': "Average trip speed (km/h)"},
    gauge={
        'axis': {'range': [min_speed, max_speed]},
        'steps': [
            {'range': [min_speed, max_speed/2], 'color': "lightgray"},
            {'range': [max_speed/2, max_speed], 'color': "gray"}]
        }))

fig.update_layout(
    font=dict(family="Arial", size=18, color="black"),
    margin=dict(t=50, b=0, l=0, r=0),
    width=400, height=400)

fig.show()

#### Trip speed distribution

In [ ]:
fig = px.histogram(df_filter, x='speedobd', nbins=10,
                   color_discrete_sequence=['#007bff'],
                   opacity=0.8,
                   title='Trip speed distribution',
                   barmode='overlay')

fig.update_layout(xaxis=dict(title=dict(font=dict(size=18))),
                  yaxis=dict(title=dict(font=dict(size=18))),
                  legend=dict(font=dict(size=16)),
                  font=dict(size=18))

fig.update_xaxes(title="Speed (km/h)")
fig.update_yaxes(title="Number of speed sensor records")

fig.show()

### Part 2) Trip time and route


#### Trip time

In [40]:
total_time = int(df_filter['timersincejourneystart'].values[-1])
td = datetime.timedelta(seconds=total_time)
d = datetime.datetime(1, 1, 1) + td
total_time_ok = f'{d.hour}h {d.minute:02}min {d.second:02}sec'

In [ ]:
h3_style = f'color: #1998cf; font-size: 22px; font-weight: bold;'
h1_style = f'color: #0b6c96; font-size: 26px; font-weight: bold;'

html = f'<h3 style="{h3_style}">Total trip time</h3> <h1 style="{h1_style}"> {total_time_ok} </h1>'

display(HTML(html))

#### Route

In [19]:
lat = df_filter['gpslatitude']
lon = df_filter['gpslongitude']

In [67]:
def get_color(speed):
  if speed < 10.0:
    return '#fdff7d'
  elif speed < 20.0: 
    return '#fcff33'
  elif speed < 30.0:
    return '#ffd333'
  elif speed < 40.0:
    return '#fab00f'
  elif speed < 50.0:
    return '#fa6d0f'
  elif speed < 60.0: 
    return '#fc5108'
  elif speed < 70.0: 
    return '#ff523b'
  elif speed < 80.0: 
    return '#ff1e00'
  elif speed < 90.0: 
    return '#ff0055'
  elif speed < 100.0: 
    return '#bd1772'
  else:
    return '#851b78'

In [ ]:
points = list(zip(lat, lon))
route_map = folium.Map(location=points[int(len(points)/2)], zoom_start=11)
polilyne = folium.PolyLine(points, color='red').add_to(route_map)

speed = df_filter['speedobd'].reset_index().drop(columns='index')
points_speed = polilyne.locations

for i, point in enumerate(points_speed):
    cor = get_color(float(speed.values[i]))
    if i > 0:
        folium.PolyLine(locations=[points_speed[i-1], point], weight=5, color=cor).add_to(route_map)

start_trip = [points[0][0], points[0][1]]
final_trip = [points[-1][0], points[-1][1]]

icon_start_trip = folium.Marker(location=start_trip, 
                                popup=folium.Popup('<h3> Início </h3>'),
                                icon=folium.Icon(color='green', prefix='fa', icon='fa-solid fa-flag-checkered'))

icon_final_trip = folium.Marker(location=final_trip, 
                                popup=folium.Popup('<h3> Chegada </h3>'),
                                icon=folium.Icon(color='red', prefix='fa', icon='fa-solid fa-flag-checkered'))

icon_start_trip.add_to(route_map)
icon_final_trip.add_to(route_map)

route_map

### Part 3) Trip distance

In [79]:
total_dist = 0
for i in range(len(points)-1):
    dist = geodesic(points[i], points[i+1]).km
    total_dist += dist

In [ ]:
h3_style = f'color: #1998cf; font-size: 22px; font-weight: bold;'
h1_style = f'color: #0b6c96; font-size: 26px; font-weight: bold;'

html = f'<h3 style="{h3_style}">Total trip distance</h3> <h1 style="{h1_style}"> {total_dist:.2f} km</h1>'

display(HTML(html))